In [1]:
def optimize(inputFile, outputFile):
    import pandas as pd
    prefs=pd.read_excel(inputFile,header=[0,1,2],sheet_name='Preferences',index_col=0)
    names=prefs.index
    shifts=prefs.columns
    shift_id=shifts.get_level_values(2)
    prefs.columns=shift_id
    requirements = pd.read_excel(inputFile, sheet_name='Requirements',index_col=0)
    
    S = prefs.columns
    I = prefs.index
    N = int((len(shift_id)) / 21)
    Nights = [n for n in range(2, (len(shift_id)-1), 3)]
    l = shift_id[-1]
    A1 = 4
    q = 6
    dshift = 21
    
    from gurobipy import Model, GRB
    mod = Model()
    x = mod.addVars(I,S,vtype=GRB.BINARY)
    Lshift = mod.addVar(vtype=GRB.INTEGER)
    Lnight = mod.addVar(vtype=GRB.INTEGER)
    Ushift = mod.addVar(vtype=GRB.INTEGER)
    Unight = mod.addVar(vtype=GRB.INTEGER)
    mod.setObjective(sum(prefs.loc[i,s]*x[i,s] for i in I for s in S) - 100*(Ushift-Lshift) - 150*(Unight-Lnight),sense=GRB.MAXIMIZE)
    for s in S:
        mod.addConstr(sum(x[i,s] for i in I) == requirements.loc[s,'persons'])
    for n in range(N):
        for i in I:
            mod.addConstr(sum(x[i,n*dshift+s] for s in range(dshift))<=q)
    for i in I:
        for s in range(1,l+1):
            mod.addConstr(x[i,s-1]+x[i,s]<=1)
    for i in I:
        for s in Nights:
            mod.addConstr(x[i,s-2]+x[i,s-1]+x[i,s+2]+x[i,s+1]<=A1*(1-x[i,s]))
    for i in I:
        mod.addConstr(x[i,l-2]+x[i,l-1]<=A1*(1-x[i,l]))
    for i in I:
        for s in S:
            mod.addConstr(x[i,s]<=prefs.loc[i,s])
    for i in I:
        mod.addConstr(Lshift<=sum(x[i,s] for s in S))
        mod.addConstr(sum(x[i,s] for s in S)<=Ushift)
    for i in I:
        mod.addConstr(Lnight<=sum(x[i,s] for s in Nights+[l]))
        mod.addConstr(sum(x[i,s] for s in Nights+[l])<=Unight)
    mod.setParam('OutputFlag',False)
    mod.optimize()
    
    schedule=pd.DataFrame('',index=names,columns=shift_id)
    for i in I:
        for s in S:
            if x[i,s].x:
                schedule.loc[i,s] = x[i,s].x
    schedule.columns=shifts

    summary=pd.Series(name='Value')
    summary['Objective']=mod.objval
    summary['Total preference score']=sum(prefs.loc[i,s]*x[i,s].x for i in I for s in S)
    summary['Shift inequality']=Ushift.x-Lshift.x
    summary['Night inequality']=Unight.x-Lnight.x
    summary

    writer=pd.ExcelWriter(outputFile,datetime_format='m/dd')
    schedule.to_excel(writer,sheet_name='Schedule')
    summary.to_excel(writer,sheet_name='Summary')
    writer.save()
    
    return

In [2]:
optimize('data.xlsx', 'output_for_data.xlsx')

Using license file C:\Users\andre\gurobi.lic
Academic license - for non-commercial use only


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


**Abstract Formulation**

**Data:** 

- $S$: the set of shifts. 
- $I$: the set of nurses.
- $l$: last shift to optimize, (a night shift)
- $Nights$: the set of nights shifts, excluding the last night shift $l$. $\{2,5,8, 11\cdots,l-3\}$
- $p_{is}$: the preference of the nurse $i$ on shift $s$          
- $r_{s}$: the number of nurses needed on shift $s$
- $N$: set of weeks to optimize, (index 0)
- $A_1$: huge arbitrary binding variable = 5
- $W$: set of shifts per week, $\{0,1,\cdots,21\}$
- $q$: is the max shifts per week for any given nurse = 6
- $d_{shift}$: number of shifts per week = 21

**Decision Variables:** 

For each nurse $i \in I$, each shift $s \in S$, let $x_{is}$ denote whether the nurse $i$ is assigned the shift $s$ (1) or not (0). (binary)

auxiliary variables: $L_{shift}$ to be the minimum number of shifts worked by any nurse and $U_{shift}$ to be the maximum number of shifts worked by any nurse

auxiliary variables: $L_{night}$ to be the minimum number of night shifts worked by any nurse and $U_{night}$ to be the maximum number of night shifts worked by any nurse

**Objective and constraints:**

$$\begin{aligned}
\text{Maximize:} && \sum_{s \in S} \sum_{i \in I} p_{is}x_{is} - 100(U_{shift} - L_{shift}) - 150(U_{night} - L_{night})\\
\text{subject to:} \\
\text{(Requirements)} && \sum_{i \in I} x_{is} & = r_{s} & \text{for each shift $s \in S$.} \\
\text{(Regulations)} && \sum_{s \in W} x_{i(n*d_{shift}+s)}  & \le q & \text{for all weeks $n \in N$, for $i \in I$, $s \in \{0,1,\cdots,21\}$.} \\
\text{(No consecutive)} && x_{i(s-1)} + x_{is} &\le 1 & \text{for each nurse $i \in I$, for $s \in \{1,2,\cdots,l\}$.}  \\
\text{(Night shift rest)} && x_{i(s-2)}+x_{i(s-1)}+x_{i(s+1)}+x_{i(s+2)} &\le A_1(1-x_{is}) & \text{for all nurses $i \in I$, for $s \in Nights$} \\
\text{(Night shift rest2)} && x_{i(l-2)}+x_{i(l-1)} &\le A_1(1-x_{il}) & \text{for all nurses $i \in I$} \\
\text{(Preference)} && x_{is} &\le p_{is} & \text{for each $s \in S$ for each $i \in I$.}  \\
\text{(Total shifts)} && L_{shift} \le \sum_{s \in S} x_{is} &\le U_{shift} & \text{for all nurses $i \in I$}  \\
\text{(Total night shifts)} && L_{night} \le \sum_{s \in Nights ∪ \{l\}} x_{is} &\le U_{night} & \text{for all nurses $i \in I$, $s \in \{2,5,8,\cdots, l\}$}
\end{aligned}$$